In [ ]:
import cv2
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!bash /content/sample_data/getModels.sh

--2022-04-18 17:20:03--  https://www.dropbox.com/s/2h2bv29a130sgrk/pose_iter_440000.caffemodel
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/2h2bv29a130sgrk/pose_iter_440000.caffemodel [following]
--2022-04-18 17:20:03--  https://www.dropbox.com/s/raw/2h2bv29a130sgrk/pose_iter_440000.caffemodel
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd06f6bdbea79f04e2311eb5727.dl.dropboxusercontent.com/cd/0/inline/Bjq6IU00lQwr8nEMCrReXbTgl-8A31GPdlFbINRNhct3HS3Y-qROYWDXyUTexUJnoIyEO8lQa73LMmToZZg3djs2vSh54-WTVPtxcF3PNF-PnAUuYhnlKs3_at2rQYpYMdBORte0QJOWkvjXa0y_dObR0NnuYRy-pjAV9R_6yuCv1w/file# [following]
--2022-04-18 17:20:04--  https://ucd06f6bdbea79f04e2311eb5727.dl.dropboxusercontent.com/cd/0/inline/Bjq6IU00lQwr8nEMCrReXbT

In [ ]:
MODE = "COCO"

if MODE is "COCO":
    protoFile = "/content/sample_data/pose/coco/pose_deploy_linevec.prototxt"
    weightsFile = "pose/coco/pose_iter_440000.caffemodel"
    nPoints = 18
    POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[11,12],[0,14],[1,14],[14,16],[15,17],[0,15],[1,15]]


In [ ]:
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

inWidth = 368
inHeight = 368

In [ ]:
from os import listdir
from os.path import isfile, join
import numpy
import cv2

mypath='/content/sample_data/images'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = numpy.empty(len(onlyfiles), dtype=object)
for n in range(0, len(onlyfiles)):
  images[n] = cv2.imread( join(mypath,onlyfiles[n]) )


In [ ]:
j=0;
for im in images:
          if im is None:
              break
          frame = im
          frameCopy = np.copy(frame)
          frameWidth =im.shape[1]
          frameHeight = im.shape[0]
          threshold = 0.1
          inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                          (0, 0, 0), swapRB=False, crop=False)

          net.setInput(inpBlob)
          output = net.forward()
          H = output.shape[2]
          W = output.shape[3] 
          points = []

          for i in range(nPoints):

    # confidence map of corresponding body's part.
              probMap = output[0, i, :, :]

    # Find global maxima of the probMap.
              minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
    
    # Scale the point to fit on the original image
              x = (frameWidth * point[0]) / W
              y = (frameHeight * point[1]) / H

              if prob > threshold : 
                cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-2, lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
                cv2.circle(frame, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

    # Add the point to the list if the probability is greater than the threshold
                points.append((int(x), int(y)))
              else :
                points.append(None)

# Draw Skeleton
          for pair in POSE_PAIRS:
              partA = pair[0]
              partB = pair[1]

              if points[partA] and points[partB]:
                cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3)

          plt.figure(figsize=[10,10])
          plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
          dir_name = "/content/sample_data/output1"
          x="amg"
          a=".png"
          plt.rcParams["savefig.directory"] = os.chdir(os.path.dirname(dir_name))
          plt.savefig("{}{}{}".format(x,j,a))
          plt.figure(figsize=[10,10])
          plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
          #where to save the figures
          dir_name = "/content/sample_data/output"
          y="img"
          plt.rcParams["savefig.directory"] = os.chdir(os.path.dirname(dir_name))
          plt.savefig("{}{}{}".format(y,j,a))
          j=j+1
          mpl.rc('figure', max_open_warning = 0)

Output hidden; open in https://colab.research.google.com to view.